In [107]:
import os

import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import librosa
import numpy as np

from sklearn.utils import shuffle
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt

import tensorflow as tf

# Global vars
RANDOM_SEED = 1337
SAMPLE_RATE = 48000
SIGNAL_LENGTH = 5 # seconds
SPEC_SHAPE = (48, 128) # height x width
FMIN = 500
FMAX = 12500
MAX_AUDIO_FILES = 1500

Step 2: load CSV file

In [108]:
train = pd

# Load metadata file
train = pd.read_csv('/kaggle/input/this-works/bioacoustics labeling - nos (2) 3.csv')

step 3: label soecies as yes or no

In [109]:
train

,audio_id,seconds,birds
0,20230117_210000.WAV,5,n
1,20230117_210000.WAV,10,n
2,20230117_210000.WAV,15,n
3,20230117_210000.WAV,20,n
4,20230117_210000.WAV,25,n
...,...,...,...
225,20230227_092400,75,n
226,20230227_092400,80,n
227,20230227_092400,85,n
228,20230227_092400,95,n


second csv for just no's

In [110]:
# Code adapted from: 
# https://www.kaggle.com/frlemarchand/bird-song-classification-using-an-efficientnet
# Make sure to check out the entire notebook.

TRAIN = train
LABELS = sorted(TRAIN.birds.unique())

# Let's see how many species and samples we have left
print('NUMBER OF SPECIES IN TRAIN DATA:', len(LABELS))
print('NUMBER OF SAMPLES IN TRAIN DATA:', len(TRAIN))

NUMBER OF SPECIES IN TRAIN DATA: 1
NUMBER OF SAMPLES IN TRAIN DATA: 230


Step 5: extracting spectrograms + Step 4: split audo into 5 second chunks

In [111]:
 def get_spectrograms(input_file, output_folder, duration=5, sr=48000):
    """
    Split an audio file into 5-second spectrograms.

    Parameters:
    - input_file (str): Path to the input audio file.
    - output_folder (str): Path to the output folder to save the spectrograms.
    - duration (int): Duration of each spectrogram in seconds (default is 5 seconds).
    - sr (int): Sampling rate (default is 22050 Hz).
    """
    window_size = 1024
    # Create the output folder if it does not exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Load the audio file
    y, sr = librosa.load(input_file, sr=None)

    # Calculate the number of samples for the specified duration
    samples_per_duration = sr * duration

    # Calculate the total number of spectrograms
    total_spectrograms = len(y) // samples_per_duration

    # Iterate through the audio file and create spectrograms
    for i in range(total_spectrograms):
        window_size = 1024
    # Create the output folder if it does not exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Load the audio file
    y, sr = librosa.load(input_file, sr=None)

    # Calculate the number of samples for the specified duration
    samples_per_duration = sr * duration

    # Calculate the total number of spectrograms
    total_spectrograms = len(y) // samples_per_duration

    # Iterate through the audio file and create spectrograms
    for i in range(total_spectrograms):
        # Extract the current segment
        segment = y[i * samples_per_duration: (i + 1) * samples_per_duration]

        # Compute the spectrogram using short-time Fourier transform (STFT)
        stft = np.abs(librosa.stft(segment, n_fft=1024))
        stft = np.log(stft + 1e-9)

        stretch = (stft - stft.min()) / (stft.max() - stft.min())
        flip = np.flip(stretch, axis=0)

        # cm = plt.get_cmap('viridis')
        cm = plt.get_cmap('plasma')
        colorized = (cm(flip) * 255.0).astype(np.uint8)[:,:,:3]
        lower = 75
        enhanced = (np.clip(colorized, lower, 255) - lower) / (255 - lower)
        enhanced = (enhanced * 255.0).astype(np.uint8)
       
        file_name = input_file.split('/')
        file_name = os.path.splitext(file_name[len(file_name) - 1])
        output_file = os.path.join(output_folder, f'spectrogram_{file_name[0]}_{i}.png')    
       
        img = Image.fromarray(enhanced)
        img = img.resize((640,350), Image.ANTIALIAS)
        # img = Image.fromarray(spectrogram(clip))
        img.save(output_file)
        img.close()

        print(f'Saved spectrogram {file_name[0]}_{i}.')

In [112]:
input_dir = '/kaggle/input/sjdhfbjsbcjwhsbfijcwe/srmp background noises'
output_folder = 'hello_sounds'
files = os.listdir(input_dir)
with tqdm(total=len(files)) as pbar:
    for row in files:
        pbar.update(1)
        #labels = TRAIN.loc[TRAIN['audio_id']== row]
        audio_file_path = os.path.join(input_dir,row)
        get_spectrograms(audio_file_path, output_folder)

  0%|          | 0/21 [00:00<?, ?it/s]

Saved spectrogram 20230227_093600_0.
Saved spectrogram 20230227_093600_1.
Saved spectrogram 20230227_093600_2.
Saved spectrogram 20230227_093600_3.
Saved spectrogram 20230227_093600_4.
Saved spectrogram 20230227_093600_5.
Saved spectrogram 20230227_093600_6.
Saved spectrogram 20230227_093600_7.
Saved spectrogram 20230227_093600_8.
Saved spectrogram 20230227_093600_9.
Saved spectrogram 20230227_093600_10.
Saved spectrogram 20230227_093600_11.
Saved spectrogram 20230227_093600_12.
Saved spectrogram 20230227_093600_13.
Saved spectrogram 20230227_093600_14.
Saved spectrogram 20230227_093600_15.
Saved spectrogram 20230227_093600_16.
Saved spectrogram 20230227_093600_17.
Saved spectrogram 20230227_093600_18.
Saved spectrogram 20230227_093600_19.
Saved spectrogram 20230227_093600_20.
Saved spectrogram 20230227_093600_21.


 10%|▉         | 2/21 [00:02<00:27,  1.45s/it]

Saved spectrogram 20230227_093600_22.
Saved spectrogram 20230227_091800_0.
Saved spectrogram 20230227_091800_1.
Saved spectrogram 20230227_091800_2.
Saved spectrogram 20230227_091800_3.
Saved spectrogram 20230227_091800_4.
Saved spectrogram 20230227_091800_5.
Saved spectrogram 20230227_091800_6.
Saved spectrogram 20230227_091800_7.
Saved spectrogram 20230227_091800_8.
Saved spectrogram 20230227_091800_9.
Saved spectrogram 20230227_091800_10.
Saved spectrogram 20230227_091800_11.
Saved spectrogram 20230227_091800_12.
Saved spectrogram 20230227_091800_13.
Saved spectrogram 20230227_091800_14.
Saved spectrogram 20230227_091800_15.
Saved spectrogram 20230227_091800_16.
Saved spectrogram 20230227_091800_17.
Saved spectrogram 20230227_091800_18.
Saved spectrogram 20230227_091800_19.
Saved spectrogram 20230227_091800_20.


 14%|█▍        | 3/21 [00:06<00:39,  2.17s/it]

...

Saved spectrogram 20230227_090800_21.
Saved spectrogram 20230227_090800_22.
Saved spectrogram 20230227_092000_0.
Saved spectrogram 20230227_092000_1.
Saved spectrogram 20230227_092000_2.
Saved spectrogram 20230227_092000_3.
Saved spectrogram 20230227_092000_4.
Saved spectrogram 20230227_092000_5.
Saved spectrogram 20230227_092000_6.
Saved spectrogram 20230227_092000_7.
Saved spectrogram 20230227_092000_8.
Saved spectrogram 20230227_092000_9.
Saved spectrogram 20230227_092000_10.
Saved spectrogram 20230227_092000_11.
Saved spectrogram 20230227_092000_12.
Saved spectrogram 20230227_092000_13.
Saved spectrogram 20230227_092000_14.
Saved spectrogram 20230227_092000_15.
Saved spectrogram 20230227_092000_16.
Saved spectrogram 20230227_092000_17.
Saved spectrogram 20230227_092000_18.
Saved spectrogram 20230227_092000_19.
Saved spectrogram 20230227_092000_20.


100%|██████████| 21/21 [00:59<00:00,  2.97s/it]

Saved spectrogram 20230227_092000_21.
Saved spectrogram 20230227_092000_22.
Saved spectrogram 20230227_092400_0.
Saved spectrogram 20230227_092400_1.
Saved spectrogram 20230227_092400_2.
Saved spectrogram 20230227_092400_3.
Saved spectrogram 20230227_092400_4.
Saved spectrogram 20230227_092400_5.
Saved spectrogram 20230227_092400_6.
Saved spectrogram 20230227_092400_7.
Saved spectrogram 20230227_092400_8.
Saved spectrogram 20230227_092400_9.
Saved spectrogram 20230227_092400_10.
Saved spectrogram 20230227_092400_11.
Saved spectrogram 20230227_092400_12.
Saved spectrogram 20230227_092400_13.
Saved spectrogram 20230227_092400_14.
Saved spectrogram 20230227_092400_15.
Saved spectrogram 20230227_092400_16.
Saved spectrogram 20230227_092400_17.
Saved spectrogram 20230227_092400_18.
Saved spectrogram 20230227_092400_19.
Saved spectrogram 20230227_092400_20.
Saved spectrogram 20230227_092400_21.


100%|██████████| 21/21 [01:01<00:00,  2.95s/it]

Saved spectrogram 20230227_092400_22.


In [113]:
def get_file_paths(directory, segments_data):
    """
    Get file paths from the specified directory.
   
    Args:
    - directory: The directory to traverse.
   
    Returns:
    - file_paths: List of file paths found in the directory.
    """
    file_paths = []
    # Walk through the directory and its subdirectories
    for i in range(len(segments_data)):
        sec = str((segments_data.seconds[i]//5) - 1)
        path = "spectrogram_" + str(segments_data.audio_id[i].split(".")[0]) + "_" + sec + ".png"
        file_path = os.path.join(directory, path)
        file_paths.append(file_path)
    return file_paths

In [114]:
import numpy as np
import cv2  # You may need to install OpenCV: pip install opencv-python

# Assuming you have a list of file paths to your spectrogram images
file_paths = get_file_paths('/kaggle/working/hello_sounds', TRAIN)
#file_paths = '../working/bioacousticsoutput/'  # List of file paths to your spectrogram images
#print (file_paths)
# Initialize an empty list to store spectrogram data
spectrogram_data = []

# Load and preprocess each spectrogram image
for file_path in file_paths:
    print(file_path)
    # Load the spectrogram image (assuming it's grayscale)
    spectrogram_img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
   
    # Resize the spectrogram image if necessary
    # spectrogram_img = cv2.resize(spectrogram_img, (938, 513))  # Resize to match model input size if necessary
   
    # Normalize the spectrogram image
    spectrogram_img = spectrogram_img.astype('float32') / 255.0
   
    spectrogram_img = spectrogram_img.reshape((350, 640, 1))
   
    # Add the spectrogram data to the list
    spectrogram_data.append(spectrogram_img)

# Convert the list of spectrogram data into a numpy array
X_train = np.array(spectrogram_data)

# Print the shape of the training data
print("Shape of X_train:", X_train.shape)

/kaggle/working/hello_sounds/spectrogram_20230117_210000_0.png
/kaggle/working/hello_sounds/spectrogram_20230117_210000_1.png
/kaggle/working/hello_sounds/spectrogram_20230117_210000_2.png
/kaggle/working/hello_sounds/spectrogram_20230117_210000_3.png
/kaggle/working/hello_sounds/spectrogram_20230117_210000_4.png
/kaggle/working/hello_sounds/spectrogram_20230117_210000_5.png
/kaggle/working/hello_sounds/spectrogram_20230117_210000_6.png
/kaggle/working/hello_sounds/spectrogram_20230117_210000_7.png
/kaggle/working/hello_sounds/spectrogram_20230117_210000_8.png
/kaggle/working/hello_sounds/spectrogram_20230117_210000_9.png
/kaggle/working/hello_sounds/spectrogram_20230117_210000_10.png
/kaggle/working/hello_sounds/spectrogram_20230117_210000_11.png
/kaggle/working/hello_sounds/spectrogram_20230117_210000_12.png
/kaggle/working/hello_sounds/spectrogram_20230117_210000_13.png
/kaggle/working/hello_sounds/spectrogram_20230117_210000_14.png
/kaggle/working/hello_sounds/spectrogram_20230117_

AttributeError: 'NoneType' object has no attribute 'astype'

In [ ]:
def get_file_paths(directory, segments_data):
    """
    Get file paths from the specified directory.
    
    Args:
    - directory: The directory to traverse.
	- segments_data: Contents of CSV file.
    
    Returns:
    - file_paths: List of file paths found in the directory.
    """
    file_paths = []
    # Walk through the directory and its subdirectories
    for i in range(len(segments_data)):
        sec = str((segments_data.seconds[i]//5) - 1)
        path = "spectrogram_" + str(segments_data.audio_id[i].split(".")[0]) + "_" + sec + ".png"
        file_path = os.path.join(directory, path)
        file_paths.append(file_path)
    return file_paths

In [ ]:
import numpy as np
import cv2

# Assuming you have a list of file paths to your spectrogram images
file_paths = get_file_paths('/kaggle/working/spectrograms', TRAIN)

# Initialize an empty list to store spectrogram data
spectrogram_data = []

# Load and preprocess each spectrogram image
for file_path in file_paths:
    print(file_path)
    # Load the spectrogram image (assuming it's grayscale)
    spectrogram_img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    
    # Resize the spectrogram image if necessary
    # spectrogram_img = cv2.resize(spectrogram_img, (938, 513))  # Resize to match model input size if necessary
    
    # Normalize the spectrogram image
    spectrogram_img = spectrogram_img.astype('float32') / 255.0
    
    spectrogram_img = spectrogram_img.reshape((513, 938, 1))
    
    # Add the spectrogram data to the list
    spectrogram_data.append(spectrogram_img)

# Convert the list of spectrogram data into a numpy array
X_train = np.array(spectrogram_data)

# Reshape the data if necessary (depends on your model input shape)
#X_train = X_train.reshape(938, 513, X_train.shape[0])  # If the model expects a 4D tensor

# Print the shape of the training data
print("Shape of X_train:", X_train.shape)

In [ ]:
# Parse all samples and add spectrograms into train data, primary_labels into label data
'/kaggle/working/spectrograms/', train_labels = [], []
with tqdm(total=len('/kaggle/working/spectrograms/')) as pbar:
    for path in TRAIN_SPECS:
        pbar.update(1)

        # Open image
        spec = Image.open(path)

        # Convert to numpy array
        spec = np.array(spec, dtype='float32')
       
        # Normalize between 0.0 and 1.0
        # and exclude samples with nan
        spec -= spec.min()
        spec /= spec.max()
        if not spec.max() == 1.0 or not spec.min() == 0.0:
            continue

        # Add channel axis to 2D array
        spec = np.expand_dims(spec, -1)

        # Add new dimension for batch size
        spec = np.expand_dims(spec, 0)

        # Add to train data
        if len(train_specs) == 0:
            train_specs = spec
        else:
            train_specs = np.vstack((train_specs, spec))

        # Add to label data
        target = np.zeros((len(LABELS)), dtype='float32')
        bird = path.split(os.sep)[-2]
        target[LABELS.index(bird)] = 1.0
        if len(train_labels) == 0:
            train_labels = target
        else:
            train_labels = np.vstack((train_labels, target))


In [ ]:
# Parse audio files and extract training samples
input_dir = '/kaggle/input/bioacustics/'
output_dir = '../working/Mohammed2/'
samples = []
audio_files = TRAIN.audio_id.unique()
with tqdm(total=len(audio_files)) as pbar:
    for row in audio_files:
        pbar.update(1)
        labels = TRAIN.loc[TRAIN['audio_id']== row]
        audio_file_path = os.path.join(input_dir,row)
        samples += get_spectrograms(audio_file_path, labels , output_dir)
       

TRAIN_SPECS = shuffle(samples, random_state=RANDOM_SEED)
print('SUCCESSFULLY EXTRACTED {} SPECTROGRAMS'.format(len(TRAIN_SPECS)))


In [ ]:
plt.figure(figsize=(15, 7))
for i in range(12):
    spec = Image.open(TRAIN_SPECS[i])
    plt.subplot(3, 4, i + 1)
    plt.title(TRAIN_SPECS[i].split(os.sep)[-1])
    plt.imshow(spec, origin='lower')

In [ ]:
# Parse all samples and add spectrograms into train data, primary_labels into label data
train_specs, train_labels = [], []
with tqdm(total=len(TRAIN_SPECS)) as pbar:
    for path in TRAIN_SPECS:
        pbar.update(1)

        # Open image
        spec = Image.open(path)

        # Convert to numpy array
        spec = np.array(spec, dtype='float32')
        
        # Normalize between 0.0 and 1.0
        # and exclude samples with nan 
        spec -= spec.min()
        spec /= spec.max()
        if not spec.max() == 1.0 or not spec.min() == 0.0:
            continue

        # Add channel axis to 2D array
        spec = np.expand_dims(spec, -1)

        # Add new dimension for batch size
        spec = np.expand_dims(spec, 0)

        # Add to train data
        if len(train_specs) == 0:
            train_specs = spec
        else:
            train_specs = np.vstack((train_specs, spec))

        # Add to label data
        target = np.zeros((len(LABELS)), dtype='float32')
        bird = path.split(os.sep)[-2]
        target[LABELS.index(bird)] = 1.0
        if len(train_labels) == 0:
            train_labels = target
        else:
            train_labels = np.vstack((train_labels, target))

In [ ]:
# Make sure your experiments are reproducible
tf.random.set_seed(RANDOM_SEED)

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(SPEC_SHAPE[0], SPEC_SHAPE[1], 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.GlobalAveragePooling2D(),

    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(len(LABELS), activation='softmax')
])

print('MODEL HAS {} PARAMETERS.'.format(model.count_params()))

In [ ]:
# Compile the model and specify optimizer, loss and metric
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.01),
              metrics=['accuracy'])

In [ ]:
# Add callbacks to reduce the learning rate if needed, early stopping, and checkpoint saving
callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                  patience=2, 
                                                  verbose=1, 
                                                  factor=0.5),
             tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                              verbose=1,
                                              patience=50),
             tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5', 
                                                monitor='val_loss',
                                                verbose=0,
                                                save_best_only=True)]

In [ ]:
# Let's train the model for a few epochs
model.fit(train_specs, 
          train_labels,
          batch_size=32,
          validation_split=0.2,
          callbacks=callbacks,
          epochs=100)

In [ ]:
# Load the best checkpoint
model = tf.keras.models.load_model('best_model.h5')

# Pick a soundscape
soundscape_path = '/kaggle/input/danieldata/20230227_092000.WAV'

# Open it with librosa
sig, rate = librosa.load(soundscape_path, sr=SAMPLE_RATE)

# Store results so that we can analyze them later
data = {'row_id': [], 'prediction': [], 'score': []}

# Split signal into 5-second chunks
# Just like we did before (well, this could actually be a seperate function)
sig_splits = []
for i in range(0, len(sig), int(SIGNAL_LENGTH * SAMPLE_RATE)):
    split = sig[i:i + int(SIGNAL_LENGTH * SAMPLE_RATE)]

    # End of signal?
    if len(split) < int(SIGNAL_LENGTH * SAMPLE_RATE):
        break

    sig_splits.append(split)
    
# Get the spectrograms and run inference on each of them
# This should be the exact same process as we used to
# generate training samples!
seconds, scnt = 0, 0
for chunk in sig_splits:
    
    # Keep track of the end time of each chunk
    seconds += 5
        
    # Get the spectrogram
    hop_length = int(SIGNAL_LENGTH * SAMPLE_RATE / (SPEC_SHAPE[1] - 1))
    mel_spec = librosa.feature.melspectrogram(y=chunk, 
                                              sr=SAMPLE_RATE, 
                                              n_fft=1024, 
                                              hop_length=hop_length, 
                                              n_mels=SPEC_SHAPE[0], 
                                              fmin=FMIN, 
                                              fmax=FMAX)

    mel_spec = librosa.power_to_db(mel_spec, ref=np.max) 

    # Normalize to match the value range we used during training.
    # That's something you should always double check!
    mel_spec -= mel_spec.min()
    mel_spec /= mel_spec.max()
    
    # Add channel axis to 2D array
    mel_spec = np.expand_dims(mel_spec, -1)

    # Add new dimension for batch size
    mel_spec = np.expand_dims(mel_spec, 0)
    
    # Predict
    p = model.predict(mel_spec)[0]
    
    # Get highest scoring species
    idx = p.argmax()
    species = LABELS[idx]
    score = p[idx]
    
    # Prepare submission entry
    data['row_id'].append(soundscape_path.split(os.sep)[-1].rsplit('_', 1)[0] + 
                          '_' + str(seconds))    
    
    # Decide if it's a "nocall" or a species by applying a threshold
    if score > 0.25:
        data['prediction'].append(species)
        scnt += 1
    else:
        data['prediction'].append('nocall')
        
    # Add the confidence score as well
    data['score'].append(score)
        
print('SOUNSCAPE ANALYSIS DONE. FOUND {} BIRDS.'.format(scnt))

In [ ]:
# Make a new data frame
results = pd.DataFrame(data, columns = ['row_id', 'prediction', 'score'])

In [ ]:
results